# 01-03 Extraktion - Musterlösung



## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql petl

In [ ]:
import petl as etl
import csv
import sys
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
# Konfiguration
base_url_quellen = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/07-ETL+DWH/data"
base_url_staging = "./"

**Hinweise zur Datenbankanbindung**

Im folgenden Block werden die Verbindungen zur Datenbank hergestellt. Normalerweise würden wir für das Staging eine einzige Zieldatenbank nutzen, in der wir verschiedene Schemata als Namensräume für die 3 Quellsysteme (Bielefeld, Mannheim, Dimensionen) nutzen würden. Die Datenbanken werden dabei als `*.sqlite`-Dateien im aktuellen Verzeichnis des Rechners angelegt, auf dem dieses Notebook ausgeführt wird.

In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_stg_bielefeld  = base_url_staging + "stg_bielefeld.sqlite"
db_path_stg_mannheim   = base_url_staging + "stg_mannheim.sqlite"
db_path_stg_dimensions = base_url_staging + "stg_dimensions.sqlite"
db_path_staging        = base_url_staging + "staging.sqlite"

db_url_stg_bielefeld  = "sqlite:///" + db_path_stg_bielefeld
db_url_stg_mannheim   = "sqlite:///" + db_path_stg_mannheim
db_url_stg_dimensions = "sqlite:///" + db_path_stg_dimensions
db_url_staging        = "sqlite:///" + db_path_staging

%sql $db_url_stg_bielefeld
%sql $db_url_stg_mannheim
%sql $db_url_stg_dimensions
%sql $db_url_staging

Da SQLite keine Schemata unterstützt, legen wir stattdessen 3 getrennte Staging-Datenbanken an (stg_bielefeld, stg_mannheim & stg_dimensions). SQLite erlaubt über den `ATTACH DATABASE`-Befehl, diese 3 Datenbanken anschließend von einer gemeinsamen Datenbank (staging) aus zu bündeln und gemeinsam darauf zuzugreifen.

Wir können  anschließend von der übergeordneten Staging-Datenbank aus gleichzeitig auf Tabellen jeder der 3 eingehängten Datenbanken zugreifen, indem wir z.B. `SELECT * FROM stg_bielefeld.faelle` schreiben (lädt die Daten der Fälle-Tabelle der Bielefelder Staging-Datenbank).

In [ ]:
%%sql $db_url_staging
ATTACH DATABASE :db_path_stg_bielefeld  AS bielefeld;
ATTACH DATABASE :db_path_stg_mannheim   AS mannheim;
ATTACH DATABASE :db_path_stg_dimensions AS dimensions;

## Aufgabe: Bielefelder Rohdatentabellen extrahieren und in Staging-Bereich laden

Nutzen Sie die gleichen Befehle wie in der Demo, um die folgenden Tabellen aus dem Quelldatenverzeichnis "bielefeld" in die Zieldatenbank stg_bielefeld zu übertragen! Legen Sie dabei die Tabellen mit passenden CREATE-Table-Statements an, überführen Sie die Daten mit den PETL-Kommandos und prüfen Sie abschießend mit einer Abfrage, ob die Daten korrekt übertragen wurden. Verwenden Sie als Tabellennamen in der Datenbank den Namen der Quelldatei ohne das ".tsv"-Suffix (also z.B. patienten.tsv => patienten).

### Tabelle: `patienten.tsv`

#### 1. Zieltabelle anlegen

Legen Sie mit einem CREATE-Table die Tabelle in der Staging-Datenbank (leer) an.

💡 *Hinweise:*

* Der Code zur Anlage der Tabelle muss in einer eigenen Zelle stehen, da der `%%sql`-Befehl für mehrzeilige SQL-Statements nicht gemeinsam mit anderen Befehlen im gleichen Block verarbeitet werden kann.
* Die Datentypen der Spalten für das `CREATE`-Statement finden Sie in der Dokumentation der Bielefelder Quelldaten.

In [ ]:
%%sql $db_url_staging
DROP TABLE IF EXISTS bielefeld.patienten;
CREATE TABLE bielefeld.patienten (
  patient_id INTEGER,
  nachname VARCHAR(50),
  vorname VARCHAR(50),
  geburtsdatum VARCHAR(10),
  sterbedatum VARCHAR(10),
  geschlecht VARCHAR(1),
  strasse VARCHAR(50),
  plz VARCHAR(5),
  ort VARCHAR(50),
  LAT FLOAT,
  LON FLOAT
);

#### 2. Daten aus Quelldatei auslesen & in Zieltabelle schreiben

Nutzen Sie die PETL-Funktionen `fromtsv()` und `todb()`, um die Quelldaten aus der Patienten-Datei zu laden und die Zieltabelle zu schreiben.

In [ ]:
table_data = etl.fromtsv(base_url_quellen + "/bielefeld/patienten.tsv")
etl.todb(table_data, db_path_stg_bielefeld, 'patienten')

#### 3. Ergebniskontrolle

Abschließend prüfen Sie mit einer `SELECT`-Abfrage, ob die Tabelle korrekt übertragen wurde.


In [ ]:
%%sql $db_url_staging
SELECT *
  FROM bielefeld.patienten
 LIMIT 10

### Tabelle `diagnosen.tsv`

Führen Sie die gleichen Schritte für die Bielefelder Quelltabelle `diagnosen.tsv` durch!

#### 1. Zieltabelle anlegen

In [ ]:
%%sql $db_url_staging
DROP TABLE IF EXISTS bielefeld.diagnosen;
CREATE TABLE bielefeld.diagnosen (
  fall_id INTEGER,
  patient_id INTEGER,
  diagnose_datum VARCHAR(10),
  diagnose_code VARCHAR(20),
  diagnose_text VARCHAR(100)
);

#### 2. Daten aus Quelldatei auslesen & in Zieltabelle schreiben

In [ ]:
table_data = etl.fromtsv(base_url_quellen + "/bielefeld/diagnosen.tsv")
etl.todb(table_data, db_path_stg_bielefeld, 'diagnosen')

#### 3. Ergebniskontrolle

In [ ]:
%%sql $db_url_staging
SELECT *
  FROM bielefeld.diagnosen
 LIMIT 10